# L1, L2 Regularization: Frequentist and Bayesian 

The Lasso, also called the LASSO (least absolute shrinkage and selection operator) is a somewhat magical technique (to me) for constrained aka regularized regression, which adds an additional parameter to the cost function of a least squares regression. This additional parameter is a multiple of the sum of absolute values of the feature coefficients: the L1 norm.

The effect is twofold:

- to force coefficient values smaller, thus controlling overfitting and making the model more generalizable
- to force weak or small coefficient values to zero, thus performing feature selection for us: the remaining non-zero coefficients are more important to the estimating power of the model

(On whiteboard)
Set parameter  λ  to constrain the sum of absolute values of  β

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df = pd.read_csv("yield_forecast.csv")

In [4]:
def format_data(df,test_size):
   
    # Targets are final grade of student
    labels = df['y_pred']
    # Drop the school and the grades from features
    df = df.drop(columns=['asd_desc','state','y_pred'],axis = 1)
    
    # One-Hot Encoding of Categorical Variables
    #df = pd.get_dummies(df)
    
    #df['y'] = list(labels)
    #most_correlated = df.corr().abs()['y'].sort_values(ascending=False)
    #print(most_correlated)
    
    # Keep correlations greater than 0.2 in absolute value
    #most_correlated = most_correlated[most_correlated >= 0.2][1:]
    
    #df = df.ix[:, most_correlated.index]
    #df = df.drop(columns = 'y')
    
    # Split into training/testing sets with 25% split
    X_train, X_test, y_train, y_test = train_test_split(df, labels, 
                                                        test_size = test_size,
                                                       random_state=42)
    
    return X_train, X_test, y_train, y_test
    

In [5]:
X_train_yield, X_test_yield, y_train_yield, y_test_yield = format_data(df,0.25)
X_train_yield.head()

,cyield,irig_flag,days_under0,dewPoint,precipAccumulation,precip,days_under_n10,days_over42,days_over32,humidity,temp_delta,temperatureMin,apparentTemperatureMin,precipIntensity
167,43.7,0,0.0,38.253084,5.011,122.0,0.0,192.0,202.0,0.593458,22.783785,42.809065,39.582570,0.002351
90,126.0,1,0.0,36.287056,0.817,103.0,0.0,200.0,214.0,0.482383,24.715794,46.114159,43.728738,0.000774
5,145.0,1,0.0,34.378551,1.254,137.0,0.0,207.0,213.0,0.531636,26.164019,40.776402,38.332290,0.001316
128,105.8,1,0.0,33.207944,7.027,142.0,0.0,195.0,214.0,0.553925,26.733879,38.655467,35.487757,0.001543
118,66.7,0,0.0,35.930607,0.458,100.0,0.0,195.0,211.0,0.491963,24.069112,45.568785,43.678551,0.001033


In [6]:
smfit = sm.OLS(y_train_yield, X_train_yield).fit()
smfit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 y_pred   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     2127.
Date:                Tue, 15 May 2018   Prob (F-statistic):          6.37e-152
Time:                        07:04:38   Log-Likelihood:                -497.25
No. Observations:                 152   AIC:                             1019.
Df Residuals:                     140   BIC:                             1055.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
cyield                     0.5085      0.037     13.885      0.000       0.436       0.581
irig_flag                 21.9086      2.135     10.262      0.000      17.688      26.129
days_under0              8.65e-13   1.06e-11      0.082      0.935      -2e-11    2.18e-11
dewPoint                  -3.3828      0.776     -4.361      0.000      -4.916      -1.849
precipAccumulation         0.0498      0.080      0.624      0.533      -0.108       0.207
precip                    -0.1072      0.035     -3.041      0.003      -0.177      -0.038
days_under_n10         -7.228e-13   1.15e-12     -0.630      0.530   -2.99e-12    1.55e-12
days_over42                0.3203      0.103      3.122      0.002       0.117       0.523
days_over32               -0.2728      0.102     -2.675      0.008      -0.474      -0.071
humidity                 114.2470     30.994      3.686      0.000      52.970     175.524
temp_delta                 0.5845      0.247      2.364      0.019       0.096       1.073
temperatureMin             2.8752      0.899      3.198      0.002       1.098       4.653
apparentTemperatureMin    -1.0454      0.932     -1.122      0.264      -2.888       0.797
precipIntensity          -89.3043    550.564     -0.162      0.871   -1177.798     999.190
==============================================================================
Omnibus:                        0.743   Durbin-Watson:                   2.190
Prob(Omnibus):                  0.690   Jarque-Bera (JB):                0.867
Skew:                           0.144   Prob(JB):                        0.648
Kurtosis:                       2.767   Cond. No.                     1.07e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.41e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [30]:
predict_ols  = smfit.predict(X_test_yield)
rmse_ols = sqrt(mean_squared_error(predict_ols, y_test_yield))


In [31]:
rmse_ols

8.424805020095407

In [32]:
from sklearn import preprocessing

continous_var_list = list(X_train_yield.columns[2:])

continous_var_list.append('cyield')


In [33]:

# scale(center) training data 
for i in continous_var_list:
    X_train_yield[i]=preprocessing.scale(X_train_yield[i].astype('float64'))

In [52]:
from sklearn.linear_model import Lasso

#sklearn uses the term alpha but it is the same in effect as lambda
lasso = Lasso(alpha = 1)
ols_lasso = lasso.fit(X_train_yield,y_train_yield)


In [53]:
print(X_train_yield.columns)

Index(['cyield', 'irig_flag', 'days_under0', 'dewPoint', 'precipAccumulation',
       'precip', 'days_under_n10', 'days_over42', 'days_over32', 'humidity',
       'temp_delta', 'temperatureMin', 'apparentTemperatureMin',
       'precipIntensity'],
      dtype='object')


In [54]:
ols_lasso.coef_

array([21.22362034, 10.62431805,  0.        , -1.90546923, -0.        ,
       -0.25399776,  0.        , -0.        , -0.        , -0.66028305,
        0.        , -0.        , -0.        ,  0.        ])

In [55]:
from sklearn.metrics import mean_squared_error
from math import sqrt
predict_lasso  = ols_lasso.predict(X_test_yield)
rmse_lasso = sqrt(mean_squared_error(predict_lasso, y_test_yield))

In [56]:
rmse_lasso

1864.3329546588554


## Bayesian Lasso Regression

In the Bayesian framework, the choice of regulariser is analogous to the choice of prior over the weights. If a Gaussian prior is used, then the Maximum a Posteriori (MAP) solution will be the same as if an L2 penalty was used.  We will see this in the Ridge section. 

It is possible to imitate the effect of the L1 regulariation in Bayesian linear regression by using a Laplacian prior for feature coefficients.In fact, when you place a Laplace prior over the parameters, the MAP solution should be identical (not merely similar) to regularization with the L1 penalty and the Laplace prior will produce an identical shrinkage effect to the L1 penalty.



## What does a laplace distribution Look like?

<img src="laplace_dist.png">

As you can see above, the Laplace distribution or 'double exponential distribution' looks like two exponential distributions back to back, (-inf, inf)
- The distribution is symmetric and centered on a location, by default at zero
- The scale parameter b acts similarly to a variance parameter, making the distribution narrower &  taller when small, and wider & shallower when large.
- The tails are heavier than a normal distribution

- The effect of using this for a prior is to encourage the marginal distribution of the feature coefficients to be close to zero. 'Strong' coefficients will move from the zero point because of the effect of the data likelihood, but 'weak' coefficients will be overly influenced by the prior distribution and remain close to zero.

Since the varaince is forced to be tighter, there is a penlization of the sum of the absolute beta-values. Let's try it out, using a narrow Laplace distribution for each coefficient;  b=0.1

In [58]:

import pymc3 as pm


# We need to add our target back into the dataframe 
X_train_yield['y'] = list(y_train_yield)
formula = 'y ~ ' + ' + '.join(['%s' % variable for variable in X_train_yield.columns[:-1]])
print(formula)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


y ~ cyield + irig_flag + days_under0 + dewPoint + precipAccumulation + precip + days_under_n10 + days_over42 + days_over32 + humidity + temp_delta + temperatureMin + apparentTemperatureMin + precipIntensity


In [62]:

with pm.Model() as mdl_lasso:

    ## Use GLM submodule for simplified model specification
    ## Betas are Laplace (for Lasso)
    ## Likelihood is Normal 
    
    priors = {"Intercept": pm.Laplace.dist(mu=0,b=0.1),
          "Regressor": pm.Laplace.dist(mu=0, b=0.1)
             }
    pm.GLM.from_formula(formula, X_train_yield
               ,family=pm.glm.families.Normal(),
               priors = priors)
   
    ## take samples using Metropolis
    trc_lasso = pm.sample(40000, step=pm.Metropolis())
    
rvs_lasso = [rv.name for rv in mdl_lasso.free_RVs]

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [sd_log__]
>Metropolis: [precipIntensity]
>Metropolis: [apparentTemperatureMin]
>Metropolis: [temperatureMin]
>Metropolis: [temp_delta]
>Metropolis: [humidity]
>Metropolis: [days_over32]
>Metropolis: [days_over42]
>Metropolis: [days_under_n10]
>Metropolis: [precip]
>Metropolis: [precipAccumulation]
>Metropolis: [dewPoint]
>Metropolis: [days_under0]
>Metropolis: [irig_flag]
>Metropolis: [cyield]
>Metropolis: [Intercept]
100%|██████████| 40500/40500 [03:45<00:00, 179.44it/s]
The number of effective samples is smaller than 10% for some parameters.


In [63]:
pm.summary(trc_lasso)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
Intercept,0.033369,0.147348,0.001307,-0.243261,0.372392,12752.112726,1.000212
cyield,0.008753,0.144545,0.001244,-0.286242,0.317488,12215.797341,0.999988
irig_flag,0.023684,0.144673,0.001200,-0.261596,0.340129,13582.094278,0.999994
days_under0,0.001703,0.140275,0.001228,-0.296166,0.293691,12795.671660,1.000011
dewPoint,-0.005811,0.141128,0.001142,-0.286644,0.302634,14326.564104,0.999988
precipAccumulation,-0.002344,0.139962,0.001232,-0.285032,0.303106,13599.872401,1.000029
precip,-0.000225,0.143678,0.001154,-0.290755,0.321854,12685.048499,1.000184
days_under_n10,-0.000827,0.143340,0.001403,-0.308703,0.296381,12484.341763,1.000030
days_over42,0.002368,0.144378,0.001394,-0.297255,0.311021,12704.563450,0.999994
days_over32,0.004131,0.140339,0.001265,-0.285340,0.299848,13383.223540,1.000093


## Ridge Regression 
This Lasso model behaves similarly to Ridge regression, which has a similar constraint on the sum of squares of the feature coefficients: the L2 norm. The effect of Ridge regression is to regularize the model by inhibiting large coefficient values and thus discouraging overfitting.

I've decided to squeeze an example of Ridge regression into this post because I wanted to demonstrate a linear regression using only the 'important' features as found by the Lasso, and also since the default behaviour of the PyMC3 glm submodule is to conduct a Ridge regression.

Set parameter  λ  to constrain the sum of squared values of  β

In [ ]:
from sklearn.linear_model import RidgeCV
ridge_cv = RidgeCV(cv=5, normalize=False,alpha=0.1)
ols_ridge = ridge_cv.fit(X_train_yield, y_train_yield)

In [ ]:
X_train_yield.columns

In [ ]:

ols_ridge.coef_

In [ ]:
predict_ridge  = ols_ridge.predict(X_test_yield)
rmse_ridge = sqrt(mean_squared_error(predict_ridge, y_test_yield))



We notice that the same coeffs that were zeroed out in Lasso have the lowest beta values here. 

Let's complete the train of thought by implementing a Ridge regression in PyMC3. It is possible to imitate the effect of the L2 regulariation in Bayesian linear regression by using a Normal prior for feature coefficients. This is the default setting for the glm submodule in PyMC3.

In [ ]:
with pm.Model() as mdl_ridge:

    ## Use GLM submodule for simplified model specification
    ## Betas are Normal (as per default settings (for Ridge)
    ## Likelihood is Normal 
    
    pm.GLM.from_formula(formula, X_train_yield)
   
    ## take samples using NUTS sampler
    trc_ridge = pm.sample(2000, step=pm.NUTS())
    
rvs_ridge = [rv.name for rv in mdl_ridge.unobserved_RVs]


In [ ]:
pm.summary(trc_ridge)